In [1]:
import numpy as np
import pandas as pd
from IPython.display import display

In [2]:
#Parametros de ejecucion del programa
nombreArchivoDataset = "./Datos en Pasos Intermedios/Dataset Reconstruido.csv"
nombreColumnaDuplicadaSalida = "Interested in buying a mobile home policy"
nombreColumnaSalida = "Number of mobile home policies"
nombreColumnaFase = "Phase"
etiquetaDatosEntrenamiento = "Training"
etiquetaDatosPrueba = "Test"

colorTextoVerde = (35, 155, 86)
colorTextoRojo = (176, 58, 46)

In [3]:
def printColouredText(text, foregroundColor = (0, 0, 0), backgroundColor = (255, 255, 255)):
    return "\033[48;2;{};{};{}m\033[38;2;{};{};{}m{}\033[38;2;0;0;0m\033[48;2;255;255;255m".format(backgroundColor[0], backgroundColor[1], backgroundColor[2], foregroundColor[0], foregroundColor[1], foregroundColor[2], text)

def buildCountingPivotTable(dataframeSource, valuesColumn, indexColumn, mainColumnToGroup, secondaryColumnToGroup):
    pivotTableWith2DetailLevels = pd.pivot_table(dataframeSource, values = valuesColumn, index = indexColumn, columns = [secondaryColumnToGroup, mainColumnToGroup], aggfunc = 'count')
    pivotTableWith2DetailLevels.fillna(0.0, inplace = True)
    pivotTableWith1DetailLevel = pd.pivot_table(dataframeSource, values = valuesColumn, index = indexColumn, columns = mainColumnToGroup, aggfunc = 'count')
    pivotTableWith1DetailLevel.fillna(0.0, inplace = True)
    pivotTableWith1DetailLevel["General Total"] = pivotTableWith1DetailLevel[pivotTableWith1DetailLevel.columns].sum(axis = 1)
    pivotTableWith1DetailLevel.rename(columns = {tableColumn: ("", tableColumn) for tableColumn in pivotTableWith1DetailLevel.columns}, inplace = True)
    fullPivotTable = pd.concat([pivotTableWith2DetailLevels, pivotTableWith1DetailLevel], axis = 1)
    
    #suma = fullPivotTable.sum()
    #suma.name = "General Total"
    #print(suma.tolist())
    
    #fullPivotTable2 = fullPivotTable.copy()
    #fullPivotTable2 = fullPivotTable2.iloc[-1:]
    #fullPivotTable2.index = ["General Total"]
    #for column in fullPivotTable2.columns:
        #fullPivotTable2[column] = [0.0]
    #fullPivotTable2.loc[1] = suma.tolist()
    
    suma = fullPivotTable.sum()
    suma.name = "General Total"
    pivotTableSummarizedByRows = pd.DataFrame(suma.transpose(), columns = fullPivotTable.columns, index = pd.Index([suma.name], name = indexColumn))
    pivotTableSummarizedByRows.loc[suma.name] = suma.tolist()
    fullPivotTable = pd.concat([fullPivotTable, pivotTableSummarizedByRows])
    #fullPivotTable = pd.concat([fullPivotTable, fullPivotTable2])
    #print(fullPivotTable.columns)
    #k = [(pair[0], pair[1], "") for pair in fullPivotTable.columns]
    #print(k)
    #fullPivotTable.rename(names = fullPivotTable.names + ["ABC"], columns = {(pair[0], pair[1]): (pair[0], pair[1], "") for pair in fullPivotTable.columns}, inplace = True)
    #print(fullPivotTable.columns)
    
    #for column in fullPivotTable.columns:
        #fullPivotTable[column] = [int(value) for value in fullPivotTable[column].tolist()]
    return fullPivotTable
    #return pivotTableSummarizedByRows

In [4]:
fullDatasetFile = pd.read_csv(nombreArchivoDataset, sep = ";", decimal = ".")
fullDatasetFile[nombreColumnaDuplicadaSalida] = fullDatasetFile[nombreColumnaSalida].tolist()
fullDatasetFile.replace({nombreColumnaDuplicadaSalida: {0: "No", 1: "Yes"}}, inplace = True)

#Filtrado de informacion para separar el dataset de entrenamiento y el dataset de prueba del dataset completo
trainingDatasetFile = fullDatasetFile[fullDatasetFile[nombreColumnaFase].str.contains(etiquetaDatosEntrenamiento)]
testDatasetFile = fullDatasetFile[fullDatasetFile[nombreColumnaFase].str.contains(etiquetaDatosPrueba)]

print("Se ha extraído la información del archivo", printColouredText(nombreArchivoDataset, colorTextoRojo))
print("Se tiene(n)", printColouredText(str(len(fullDatasetFile.index)) + " dato(s)", colorTextoVerde))
print("Se tiene(n) de", printColouredText(str(len(trainingDatasetFile.index)) + " dato(s) de entrenamiento", colorTextoVerde))
print("Se tiene(n) de", printColouredText(str(len(testDatasetFile.index)) + " dato(s) de prueba", colorTextoVerde))

Se ha extraído la información del archivo ./Datos en Pasos Intermedios/Dataset Reconstruido.csv
Se tiene(n) 9822 dato(s)
Se tiene(n) de 5822 dato(s) de entrenamiento
Se tiene(n) de 4000 dato(s) de prueba


In [5]:
fullDatasetFile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9822 entries, 0 to 9821
Data columns (total 88 columns):
 #   Column                                            Non-Null Count  Dtype 
---  ------                                            --------------  ----- 
 0   Customer Subtype                                  9822 non-null   int64 
 1   Number of houses                                  9822 non-null   int64 
 2   Avg size household                                9822 non-null   int64 
 3   Avg age                                           9822 non-null   int64 
 4   Customer main type                                9822 non-null   int64 
 5   Roman catholic                                    9822 non-null   int64 
 6   Protestant                                        9822 non-null   int64 
 7   Other religion                                    9822 non-null   int64 
 8   No religion                                       9822 non-null   int64 
 9   Married                       

In [6]:
pivotTablesList = {}

for nombreColumna in fullDatasetFile.columns:
    if nombreColumna != nombreColumnaSalida and nombreColumna != nombreColumnaDuplicadaSalida and nombreColumna != nombreColumnaFase:
        pivotTablesList[nombreColumna] = buildCountingPivotTable(fullDatasetFile, nombreColumnaSalida, nombreColumna, nombreColumnaDuplicadaSalida, nombreColumnaFase)
        print("pivotTablesList[\"" + nombreColumna + "\"] = buildCountingPivotTable(fullDatasetFile, nombreColumnaSalida, \"" + nombreColumna + "\", nombreColumnaDuplicadaSalida, nombreColumnaFase)")
        print("display(pivotTablesList[\"" + nombreColumna + "\"])\n")

#pivotTablesList[] = buildCountingPivotTable(fullDatasetFile, nombreColumnaSalida, col, nombreColumnaDuplicadaSalida, nombreColumnaFase)

#display(pivotTable1)

pivotTablesList["Customer Subtype"] = buildCountingPivotTable(fullDatasetFile, nombreColumnaSalida, "Customer Subtype", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Customer Subtype"])

pivotTablesList["Number of houses"] = buildCountingPivotTable(fullDatasetFile, nombreColumnaSalida, "Number of houses", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Number of houses"])

pivotTablesList["Avg size household"] = buildCountingPivotTable(fullDatasetFile, nombreColumnaSalida, "Avg size household", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Avg size household"])

pivotTablesList["Avg age"] = buildCountingPivotTable(fullDatasetFile, nombreColumnaSalida, "Avg age", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Avg age"])

pivotTablesList["Customer main type"] = buildCountingPivotTable(fullDatasetFile, nombreColumnaSalida, "Customer main type", nombreColumnaDuplicadaSalida, nombreC

pivotTablesList["Income 75-122.000"] = buildCountingPivotTable(fullDatasetFile, nombreColumnaSalida, "Income 75-122.000", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Income 75-122.000"])

pivotTablesList["Income >123.000"] = buildCountingPivotTable(fullDatasetFile, nombreColumnaSalida, "Income >123.000", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Income >123.000"])

pivotTablesList["Average income"] = buildCountingPivotTable(fullDatasetFile, nombreColumnaSalida, "Average income", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Average income"])

pivotTablesList["Purchasing power class"] = buildCountingPivotTable(fullDatasetFile, nombreColumnaSalida, "Purchasing power class", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Purchasing power class"])

pivotTablesList["Contribution private third party insurance"] = buildCountingPivotTable(fullDatasetFile, nombreColumnaSalida, "C

pivotTablesList["Number of bicycle policies"] = buildCountingPivotTable(fullDatasetFile, nombreColumnaSalida, "Number of bicycle policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Number of bicycle policies"])

pivotTablesList["Number of property insurance policies"] = buildCountingPivotTable(fullDatasetFile, nombreColumnaSalida, "Number of property insurance policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Number of property insurance policies"])

pivotTablesList["Number of social security insurance policies"] = buildCountingPivotTable(fullDatasetFile, nombreColumnaSalida, "Number of social security insurance policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Number of social security insurance policies"])



In [7]:
pivotTablesList["Customer Subtype"] = buildCountingPivotTable(fullDatasetFile, nombreColumnaSalida, "Customer Subtype", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Customer Subtype"])

Phase                                     Phase 1 [Training]         \
Interested in buying a mobile home policy                 No    Yes   
Customer Subtype                                                      
1                                                      111.0   13.0   
2                                                       76.0    6.0   
3                                                      224.0   25.0   
4                                                       50.0    2.0   
5                                                       43.0    2.0   
6                                                      107.0   12.0   
7                                                       41.0    3.0   
8                                                      288.0   51.0   
9                                                      266.0   12.0   
10                                                     156.0    9.0   
11                                                     144.0    9.0   
12                                                      95.0   16.0   
13                                                     166.0   13.0   
15                                                       5.0    0.0   
16                                                      16.0    0.0   
17                                                       9.0    0.0   
18                                                      19.0    0.0   
19                                                       3.0    0.0   
20                                                      23.0    2.0   
21                                                      15.0    0.0   
22                                                      94.0    4.0   
23                                                     247.0    4.0   
24                                                     175.0    5.0   
25                                                      80.0    2.0   
26                                                      47.0    1.0   
27                                                      49.0    1.0   
28                                                      25.0    0.0   
29                                                      84.0    2.0   
30                                                     114.0    4.0   
31                                                     199.0    6.0   
32                                                     133.0    8.0   
33                                                     764.0   46.0   
34                                                     173.0    9.0   
35                                                     206.0    8.0   
36                                                     209.0   16.0   
37                                                     122.0   10.0   
38                                                     316.0   23.0   
39                                                     309.0   19.0   
40                                                      71.0    0.0   
41                                                     200.0    5.0   
General Total                                         5474.0  348.0   

Phase                                     Phase 2 [Test]                 \
Interested in buying a mobile home policy             No    Yes      No   
Customer Subtype                                                          
1                                                   81.0   13.0   192.0   
2                                                   61.0    5.0   137.0   
3                                                  176.0    8.0   400.0   
4                                                   37.0    1.0    87.0   
5                                                   25.0    0.0    68.0   
6                                                   76.0   14.0   183.0   
7                                                   26.0    2.0    67.0   
8                                                  186.0   21.0   474.0   
9                                                 

In [8]:
pivotTablesList["Number of houses"] = buildCountingPivotTable(fullDatasetFile, nombreColumnaSalida, "Number of houses", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Number of houses"])

Phase                                     Phase 1 [Training]         \
Interested in buying a mobile home policy                 No    Yes   
Number of houses                                                      
1                                                     4952.0  315.0   
2                                                      472.0   33.0   
3                                                       39.0    0.0   
4                                                        2.0    0.0   
5                                                        1.0    0.0   
6                                                        1.0    0.0   
7                                                        5.0    0.0   
8                                                        1.0    0.0   
10                                                       1.0    0.0   
General Total                                         5474.0  348.0   

Phase                                     Phase 2 [Test]                 \
Interested in buying a mobile home policy             No    Yes      No   
Number of houses                                                          
1                                                 3437.0  211.0  8389.0   
2                                                  290.0   26.0   762.0   
3                                                   24.0    1.0    63.0   
4                                                    2.0    0.0     4.0   
5                                                    2.0    0.0     3.0   
6                                                    2.0    0.0     3.0   
7                                                    3.0    0.0     8.0   
8                                                    1.0    0.0     2.0   
10                                                   1.0    0.0     2.0   
General Total                                     3762.0  238.0  9236.0   

Phase                                                           
Interested in buying a mobile home policy    Yes General Total  
Number of houses                                                
1                                          526.0        8915.0  
2                                           59.0         821.0  
3                                            1.0          64.0  
4                                            0.0           4.0  
5                                            0.0           3.0  
6                                            0.0           3.0  
7                                            0.0           8.0  
8                                            0.0           2.0  
10                                           0.0           2.0  
General Total                              586.0        9822.0

In [9]:
pivotTablesList["Avg size household"] = buildCountingPivotTable(fullDatasetFile, nombreColumnaSalida, "Avg size household", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Avg size household"])

Phase                                     Phase 1 [Training]         \
Interested in buying a mobile home policy                 No    Yes   
Avg size household                                                    
1                                                      276.0    8.0   
2                                                     2016.0  115.0   
3                                                     2475.0  171.0   
4                                                      643.0   50.0   
5                                                       64.0    4.0   
6                                                        0.0    0.0   
General Total                                         5474.0  348.0   

Phase                                     Phase 2 [Test]                 \
Interested in buying a mobile home policy             No    Yes      No   
Avg size household                                                        
1                                                  165.0    3.0   441.0   
2                                                 1405.0   80.0  3421.0   
3                                                 1763.0  104.0  4238.0   
4                                                  391.0   48.0  1034.0   
5                                                   35.0    3.0    99.0   
6                                                    3.0    0.0     3.0   
General Total                                     3762.0  238.0  9236.0   

Phase                                                           
Interested in buying a mobile home policy    Yes General Total  
Avg size household                                              
1                                           11.0         452.0  
2                                          195.0        3616.0  
3                                          275.0        4513.0  
4                                           98.0        1132.0  
5                                            7.0         106.0  
6                                            0.0           3.0  
General Total                              586.0        9822.0

In [10]:
pivotTablesList["Avg age"] = buildCountingPivotTable(fullDatasetFile, nombreColumnaSalida, "Avg age", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Avg age"])

Phase                                     Phase 1 [Training]         \
Interested in buying a mobile home policy                 No    Yes   
Avg age                                                               
1                                                       73.0    1.0   
2                                                     1365.0   87.0   
3                                                     2817.0  183.0   
4                                                     1009.0   64.0   
5                                                      181.0   12.0   
6                                                       29.0    1.0   
General Total                                         5474.0  348.0   

Phase                                     Phase 2 [Test]                 \
Interested in buying a mobile home policy             No    Yes      No   
Avg age                                                                   
1                                                   30.0    0.0     103   
2                                                  888.0   69.0    2253   
3                                                 2034.0  120.0    4851   
4                                                  663.0   41.0    1672   
5                                                  128.0    8.0     309   
6                                                   19.0    0.0      48   
General Total                                     3762.0  238.0  9236.0   

Phase                                                           
Interested in buying a mobile home policy    Yes General Total  
Avg age                                                         
1                                              1           104  
2                                            156          2409  
3                                            303          5154  
4                                            105          1777  
5                                             20           329  
6                                              1            49  
General Total                              586.0        9822.0

In [11]:
pivotTablesList["Customer main type"] = buildCountingPivotTable(fullDatasetFile, nombreColumnaSalida, "Customer main type", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Customer main type"])

Phase                                     Phase 1 [Training]         \
Interested in buying a mobile home policy                 No    Yes   
Customer main type                                                    
1                                                      504.0   48.0   
2                                                      436.0   66.0   
3                                                      827.0   59.0   
4                                                       52.0    0.0   
5                                                      554.0   15.0   
6                                                      201.0    4.0   
7                                                      530.0   20.0   
8                                                     1474.0   89.0   
9                                                      625.0   42.0   
10                                                     271.0    5.0   
General Total                                         5474.0  348.0   

Phase                                     Phase 2 [Test]                 \
Interested in buying a mobile home policy             No    Yes      No   
Customer main type                                                        
1                                                  380.0   27.0   884.0   
2                                                  288.0   37.0   724.0   
3                                                  577.0   50.0  1404.0   
4                                                   27.0    0.0    79.0   
5                                                  368.0    3.0   922.0   
6                                                  116.0    5.0   317.0   
7                                                  316.0   15.0   846.0   
8                                                 1069.0   62.0  2543.0   
9                                                  411.0   33.0  1036.0   
10                                                 210.0    6.0   481.0   
General Total                                     3762.0  238.0  9236.0   

Phase                                                           
Interested in buying a mobile home policy    Yes General Total  
Customer main type                                              
1                                           75.0         959.0  
2                                          103.0         827.0  
3                                          109.0        1513.0  
4                                            0.0          79.0  
5                                           18.0         940.0  
6                                            9.0         326.0  
7                                           35.0         881.0  
8                                          151.0        2694.0  
9                                           75.0        1111.0  
10                                          11.0         492.0  
General Total                              586.0        9822.0

In [12]:
pivotTableWith2DetailLevels = pd.pivot_table(fullDatasetFile, values = nombreColumnaSalida, index = "Customer Subtype", columns = [nombreColumnaFase, nombreColumnaDuplicadaSalida], aggfunc = 'count')
pivotTableWith2DetailLevels.fillna(0.0, inplace = True)
pivotTableWith1DetailLevel = pd.pivot_table(fullDatasetFile, values = nombreColumnaSalida, index = "Customer Subtype", columns = nombreColumnaDuplicadaSalida, aggfunc = 'count')
pivotTableWith1DetailLevel.fillna(0.0, inplace = True)
pivotTableWith1DetailLevel["General Total"] = pivotTableWith1DetailLevel[pivotTableWith1DetailLevel.columns].sum(axis = 1)
pivotTableWith1DetailLevel.rename(columns = {tableColumn: ("", tableColumn) for tableColumn in pivotTableWith1DetailLevel.columns}, inplace = True)
fullPivotTable = pd.concat([pivotTableWith2DetailLevels, pivotTableWith1DetailLevel], axis = 1)

suma = fullPivotTable.sum()
suma.name = "General Total"
pivotTableSummarizedByRows = pd.DataFrame(suma.transpose(), columns = fullPivotTable.columns, index = [suma.name])
pivotTableSummarizedByRows.loc[suma.name] = suma.tolist()
fullPivotTable = pd.concat([fullPivotTable, pivotTableSummarizedByRows])

for column in fullPivotTable.columns:
    fullPivotTable[column] = [int(value) for value in fullPivotTable[column].tolist()]

#x1.info()
display(fullPivotTable)

Phase                                     Phase 1 [Training]       \
Interested in buying a mobile home policy                 No  Yes   
1                                                        111   13   
2                                                         76    6   
3                                                        224   25   
4                                                         50    2   
5                                                         43    2   
6                                                        107   12   
7                                                         41    3   
8                                                        288   51   
9                                                        266   12   
10                                                       156    9   
11                                                       144    9   
12                                                        95   16   
13                                                       166   13   
15                                                         5    0   
16                                                        16    0   
17                                                         9    0   
18                                                        19    0   
19                                                         3    0   
20                                                        23    2   
21                                                        15    0   
22                                                        94    4   
23                                                       247    4   
24                                                       175    5   
25                                                        80    2   
26                                                        47    1   
27                                                        49    1   
28                                                        25    0   
29                                                        84    2   
30                                                       114    4   
31                                                       199    6   
32                                                       133    8   
33                                                       764   46   
34                                                       173    9   
35                                                       206    8   
36                                                       209   16   
37                                                       122   10   
38                                                       316   23   
39                                                       309   19   
40                                                        71    0   
41                                                       200    5   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested in buying a mobile home policy             No  Yes    No  Yes   
1                                                     81   13   192   26   
2                                                     61    5   137   11   
3                                                    176    8   400   33   
4                                                     37    1    87    3   
5                                                     25    0    68    2   
6                                                     76   14   183   26   
7                                                     26    2    67    5   
8                                                    186   21   474   72   
9                                                    177    5   443   17   
10                                                    92   14   248   23   
11                                                   126    7   270   16   
12                                          